# LALSimulation
This uses LALSimulation. The reason this has been put on pause as ```XLAL Error - XLALSimIMRSpinAlignedEOBModes: Ringdown frequency > Nyquist frequency!``` is an error that keeps popping up because I'm struggling to actually change the parameter_check() values.

We're using OOP to create a LALCompactBinaryCoalescenseGenerator object: https://lscsoft.docs.ligo.org/lalsuite/lalsimulation/classlalsimulation_1_1gwsignal_1_1core_1_1waveform_1_1_l_a_l_compact_binary_coalescence_generator.html

In [2]:
%matplotlib inline

import lal
import lalsimulation
import lalsimulation.gwsignal.core.waveform as wv
from astropy import units as u
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'lal'

In [ ]:
from pycbc.waveform import td_approximants, fd_approximants
from pycbc.waveform import get_td_waveform

In [ ]:
def whiten(strain, interp_psd, dt):
    Nt = len(strain)
    freqs = np.fft.rfftfreq(Nt, dt)
    hf = np.fft.rfft(strain)
    white_hf = hf / (np.sqrt(interp_psd(freqs) /dt/2.))
    white_ht = np.fft.irfft(white_hf, n=Nt)
    return white_ht

In [ ]:
j = 1
hp, hc = get_td_waveform(approximant=labels[j], mass1=m1[j], mass2=m2[j], delta_t=1.0/4096,
                         f_lower=20, distance=100, f_ref=20, delta_f=1./4, f_final = 2048)
def preprocessing(data, time):
    fs = 4096
    NFFT = 1*fs
    pxx, freq = mlab.psd(data, Fs = fs, NFFT = NFFT) #obtains transformed signal via welch's psd
    psd = interp1d(freq, pxx) # constructs the interpolated function
    dt = time[1]-time[0] # time binning
    whitened_SEOBNRv4 = whiten(data, psd, dt)

    bb, ab = butter(4, [20.*2./fs, 300.*2./fs], btype='band')
    strain_SEOBNRv4 = filtfilt(bb, ab, whitened_SEOBNRv4)
    strain_L1_shift = np.roll(whitened_SEOBNRv4,int(fs))

processed = preprocessing(hp, hp.sample_times)

In [ ]:
import h5py

# LIGO-specific readligo.py 
import readligo as rl

In [ ]:
fn_H1 = '/home/brett/Downloads/H-H1_GWOSC_O3b_4KHZ_R1-1256697856-4096.hdf5'
strain_H1, time_H1, chan_dict_H1 = rl.loaddata(fn_H1, 'H1')

fs = 4096
time = time_H1
dt = time[1] - time[0]

def whiten(strain, interp_psd, dt):
    Nt = len(strain)
    freqs = np.fft.rfftfreq(Nt, dt)

    # whitening: transform to freq domain, divide by asd, then transform back, 
    # taking care to get normalization right.
    hf = np.fft.rfft(strain)
    white_hf = hf / (np.sqrt(interp_psd(freqs) /dt/2.))
    white_ht = np.fft.irfft(white_hf, n=Nt)
    return white_ht

NFFT = 1*fs
fmin = 10
fmax = 2000
tevent = (time[-1]-time[0])/2+time[0]

Pxx_H1, freqs = mlab.psd(strain_H1, Fs = fs, NFFT = NFFT)
# We will use interpolations of the ASDs computed above for whitening:
psd_H1 = interp1d(freqs, Pxx_H1)

strain_H1_whiten = whiten(strain_H1,psd_H1,dt)
bb, ab = butter(4, [20.*2./fs, 300.*2./fs], btype='band')
strain_H1_whitenbp = filtfilt(bb, ab, strain_H1_whiten)

tevent = (time[-1]-time[0])/2+time[0]

In [ ]:
#%matplotlib inline

import lal
import lalsimulation
import lalsimulation.gwsignal.core.waveform as wv
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import numpy as np
import h5py
import random
import scipy.signal as sg

from astropy import units as u
from scipy.interpolate import interp1d
from scipy.signal import butter, filtfilt, iirdesign, zpk2tf, freqz, medfilt
import readligo as rl
from gwpy.timeseries import TimeSeries
from scipy import integrate

In [ ]:
labels = ['SEOBNRv4', 'TaylorT2', 'SEOBNRv4_ROM_NRTidalv2_NSBH']
mass1 = [2, 1, 19.5] #varying masses based on aprox we're using. for i = 3, m1 >= m2
mass2 = [95, 2, 1.5]

fmin = 20
fref = 20
fmax = 2000
fn_H1 = '/home/brett/Downloads/H-H1_GWOSC_O3b_4KHZ_R1-1256697856-4096.hdf5'
num_of_waveform = 100

# we need to figure out all these values
dt = 1./4096
df = 1./4
dist = 100
fs = 4096 #I think this should be 4096, but if i put it that i get ringdown freq > nyquist freq
NFFT = 1*fs

In [ ]:
lalsim_data = [[], []]

if __name__=='__main__':
    approxs = [lalsimulation.SEOBNRv4, lalsimulation.TaylorT2]#, lalsimulation.SEOBNRv4_ROM_NRTidalv2_NSBH]
    parentWV = wv.CompactBinaryCoalescenceGenerator()
    
    for i in [0, 1]:
        while len(lalsim_data[i]) < num_of_waveform:
            m1 = random.randint(mass1[i], mass2[i])
            m2 = random.randint(mass1[i], mass2[i])
            while m1/m2 > 10:
                m2 = random.randint(2,95)
            waveform = wv.LALCompactBinaryCoalescenceGenerator(approxs[i])
            lalsim_data[i].append(waveform.generate_td_waveform(mass1 = m1*u.solMass,
                                                             mass2 = m2*u.solMass,
                                                             deltaT = dt*u.s, 
                                                             deltaF = df*u.Hz,
                                                             distance = dist*u.Mpc,
                                                             f_max = fmax*u.Hz))

In [ ]:
def whiten(strain, interp_psd, dt):
    Nt = len(strain)
    freqs = np.fft.rfftfreq(Nt, dt)
    hf = np.fft.rfft(strain)
    white_hf = hf / (np.sqrt(interp_psd(freqs) /dt/2.))
    white_ht = np.fft.irfft(white_hf, n=Nt)
    return white_ht

def preprocessing(data, dt, t0):
    pxx, freq = mlab.psd(data, Fs = fs, NFFT = len(data)) #obtains transformed signal via welch's psd
    psd = interp1d(freq, pxx) # constructs the interpolated function
    whitened_SEOBNRv4 = whiten(data, psd, dt)
    
    #bb, ab = butter(4, [20.*2./fs, 300.*2./fs], btype='band')
    #strain_SEOBNRv4 = filtfilt(bb, ab, whitened_SEOBNRv4)
    
    shift = -(t0/dt)
    strain_L1_shift = np.roll(whitened_SEOBNRv4,int(shift))
    return strain_L1_shift

In [ ]:
def h_tilde(f, i):
    idx = i  # Waveform index
    waveform_data = processed_SEOBNRv4[idx] # Waveform data from SEOBNRv4 index
    
    # FT data
    h_tilde_data = np.fft.rfft(waveform_data)
    
    # Interpolate to evaluate at frequency
    h_tilde_interp = interp1d(np.fft.rfftfreq(len(waveform_data), dt), h_tilde_data)
    
    return h_tilde_interp

def optimal_snr_squared(h_tilde, psd):
    def integrand(f):
        return np.abs(h_tilde(f))**2 / psd(f)

    result, _ = integrate.quad(integrand, fmin, fmax)
    rho_opt_squared = 4 * result

    return rho_opt_squared

from pycbc.filter import matched_filter

def snr(data, conditioned, psd):
    snr = matched_filter(data, conditioned, psd=psd, low_frequency_cutoff=20)
    snr = snr.crop(4 + 4, 4)
    
    pylab.figure(figsize=[10, 4])
    pylab.plot(snr.sample_times, abs(snr))
    pylab.ylabel('Signal-to-noise')
    pylab.xlabel('Time (s)')
    pylab.show()

    peak = abs(snr).numpy().argmax()
    snrp = snr[peak]
    time = snr.sample_times[peak]

    print("We found a signal at {}s with SNR {}".format(time, 
                                                        abs(snrp)))

In [ ]:
processed_seo = []
processed_taylor = []

for i in range(num_of_waveform):
    proc = preprocessing(lalsim_data[0][i][0], lalsim_data[0][i][0].dt.value, lalsim_data[0][i][0].t0.value)
    processed_seo.append(proc)
    
for i in range(num_of_waveform):
    proc = preprocessing(lalsim_data[1][i][0], lalsim_data[1][i][0].dt.value, lalsim_data[1][i][0].t0.value)
    processed_taylor.append(proc)

In [ ]:
H1s = ['/home/brett/Downloads/H-H1_GWOSC_O3b_4KHZ_R1-1256697856-4096.hdf5']
H1_noise = []
for fn_H1 in H1s:
    strain_H1, time_H1, chan_dict_H1 = rl.loaddata(fn_H1, 'H1')

    time = time_H1
    dt = time[1] - time[0]
    tevent = (time[-1]-time[0])/2+time[0]
    
    strain_H1_preprocess = preprocessing(strain_H1, dt, -dt)
    bb, ab = butter(4, [20.*2./fs, 300.*2./fs], btype='band')
    strain_H1_whitenbp = filtfilt(bb, ab, strain_H1_whiten)
    H1_noise.append(strain_H1_whitenbp)

In [ ]:
j = 10
final_seo = []
final_taylor = []

scale_seo = []
scale_taylor = []

for j in range(num_of_waveform):
    #snr = optimal_snr_squared(h_tilde(j), psd_H1)
    scale_seo.append(1.5*processed_seo[j]/np.max(processed_seo[j]))
    scale_taylor.append(1.5*processed_taylor[j]/np.max(processed_taylor[j]))

    center = int(len(strain_H1_whitenbp)/2)
    final_seo.append(np.add(strain_H1_whitenbp[center:center+len(scale_seo[j])], scale_seo[j]))
    final_taylor.append(np.add(strain_H1_whitenbp[center:center+len(scale_taylor[j])], scale_taylor[j]))

tevent = (time[-1]-time[0])/2+time[0]

In [ ]:
final_taylor_np = np.array([])
for element in final_taylor:
    final_taylor_np = np.append(final_taylor_np, element.value)

final_seo_np = np.array([])
for element in final_taylor:
    final_seo_np = np.append(final_seo_np, element.value)

In [ ]:
np.savez('/home/brett/Desktop/final_taylor_data.npz', final_taylor_np, delimiter=',')
np.savez('/home/brett/Desktop/final_seo_data.npz', final_seo_np, delimiter=',')